# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [21]:
!pip install ultralytics

In [22]:
import cv2
from ultralytics import YOLO, solutions
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm



## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [23]:
model = YOLO('/content/drive/MyDrive/Project/YOLO_Model.pt')

In [24]:
# cap = cv2.VideoCapture("/content/drive/MyDrive/Project/YOLO_Video.mp4")
# assert cap.isOpened(), "Error reading video file"

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [25]:
video_path = '/content/drive/MyDrive/Project/YOLO_Video.mp4'
video = cv2.VideoCapture(video_path)
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)


In [26]:
output_path = '/content/drive/MyDrive/Project/YOLO_Video_output.mp4'
VIDEO_CODEC = "MP4V"
output_video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*VIDEO_CODEC), fps, (width, height))


In [27]:
# Define Regions of Interest (ROIs) for counting vehicles
horizontal_line_up = [(0, int(height * 0.25)), (width, int(height * 0.25))]  # 25% from the top
horizontal_line_down = [(0, int(height * 0.75)), (width, int(height * 0.75))]  # 75% from the top
vertical_line_left = [(int(width * 0.25), 0), (int(width * 0.25), height)]  # 25% from the left
vertical_line_right = [(int(width * 0.75), 0), (int(width * 0.75), height)]  # 75% from the left

In [28]:
class_IDS = [2, 3, 5, 7]
veiculos_contador_in = dict.fromkeys(class_IDS, 0)
veiculos_contador_out = dict.fromkeys(class_IDS, 0)
contador_in = 0
contador_out = 0

In [29]:
# Define additional lines and counters
cy_linha_up = int(height * 0.25)
cy_linha_down = int(height * 0.75)
cx_linha_left = int(width * 0.25)
cx_linha_right = int(width * 0.75)
offset = 4

In [30]:
contador_top = 0
contador_bottom = 0
contador_left = 0
contador_right = 0

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [31]:
# # Video writer
# video_writer = cv2.VideoWriter("object_counting_output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [32]:
# # Combine all lines into a single list
# all_lines = [
#     horizontal_line_up,
#     horizontal_line_down,
#     vertical_line_left,
#     vertical_line_right,
# ]


In [33]:
# # Convert tuples in all_lines to lists of lists
# all_lines = [[list(point) for point in line] for line in all_lines]
# # Flatten the list of lists
# all_lines = [item for sublist in all_lines for item in sublist]

# counter = solutions.ObjectCounter(
#     view_img=True,
#     reg_pts=all_lines,  # Pass the list of all lines
#     names=model.names,
#     draw_tracks=True,
#     line_thickness=2,
# )

In [34]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [35]:
device

'cpu'

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [36]:
for _ in tqdm(range(int(video.get(cv2.CAP_PROP_FRAME_COUNT)))):
    ret, frame = video.read()
    if not ret:
        break

    # Perform object detection using YOLOv8
    results = model.predict(frame, conf=0.5, device='cpu', verbose=False)

    detections = results[0].boxes.xyxy.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    positions_frame = pd.DataFrame(detections, columns=['xmin', 'ymin', 'xmax', 'ymax'])
    positions_frame['conf'] = confidences
    positions_frame['class'] = classes

    # Draw lines on the frame
    cv2.line(frame, (0, cy_linha_up), (width, cy_linha_up), (255, 0, 0), 2)
    cv2.line(frame, (0, cy_linha_down), (width, cy_linha_down), (255, 0, 0), 2)
    cv2.line(frame, (cx_linha_left, 0), (cx_linha_left, height), (255, 0, 0), 2)
    cv2.line(frame, (cx_linha_right, 0), (cx_linha_right, height), (255, 0, 0), 2)

    for _, row in positions_frame.iterrows():
        xmin, ymin, xmax, ymax, conf, category = row
        xmin, ymin, xmax, ymax, category = int(xmin), int(ymin), int(xmax), int(ymax), int(category)
        center_x, center_y = (xmin + xmax) // 2, (ymin + ymax) // 2

        # Draw bounding box and display confidence and class ID
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
        label = f'Class:{category} Conf:{conf:.2f}'
        cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # Count vehicles crossing lines
        if abs(center_y - cy_linha_up) <= offset:
            contador_top += 1
        elif abs(center_y - cy_linha_down) <= offset:
            contador_bottom += 1
        if abs(center_x - cx_linha_left) <= offset:
            contador_left += 1
        elif abs(center_x - cx_linha_right) <= offset:
            contador_right += 1

    # Display count of vehicles crossing each line
    cv2.putText(frame, f'Crossing Up: {contador_top}', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(frame, f'Crossing Down: {contador_bottom}', (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(frame, f'Crossing Left: {contador_left}', (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(frame, f'Crossing Right: {contador_right}', (30, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Write the frame to the output video
    output_video.write(frame)

video.release()
output_video.release()
cv2.destroyAllWindows()

  0%|          | 0/806 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [39]:
video.release()
output_video.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video